In [449]:
import nltk
import random
import spacy
import en_core_web_sm
import pandas as pd
import numpy as np
nlp = en_core_web_sm.load()
from nltk.tokenize import sent_tokenize, word_tokenize

from nltk.corpus import stopwords

## Sentiment Analysis 

In [89]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [90]:
analyzer = SentimentIntensityAnalyzer()

In [450]:
analyzer.polarity_scores('Awesome') # sample string for testing

{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.6249}

In [92]:
!pip install PyPDF2

  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61087 sha256=b047f5514ccc85db424d9dda2143f7cb3f276bd26ec5e2b235ef3c4988909979
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\b1\1a\8f\a4c34be976825a2f7948d0fa40907598d69834f8ab5889de11
Successfully built PyPDF2


In [93]:
import PyPDF2

In [94]:
file = open('Feedback Categories and Samples.pdf','rb')


In [100]:
pdfreader = PyPDF2.PdfFileReader(file)
type(pdfreader.numPages)

int

In [96]:
page = pdfreader.getPage(0)

In [98]:
string = page.extractText()

In [126]:
jreview = open('Johnreview.txt','a+')

for i in range(0,pdfreader.numPages):
    page = pdfreader.getPage(i)
    string = page.extractText()
    jreview.write(string)

jreview.close()
    
    

In [127]:
jread = open('Johnreview.txt','r+')
string = jread.read()

In [128]:
str_list = []


In [131]:
lst_str = string.split('\n\n')

In [132]:
len(lst_str)

49

In [133]:
for i in range(0,len(lst_str)):
#     lst_str[i].replace('\n','')
    print(lst_str[i])


John demonstrates adequate communication skills and completes all requested tasks and required 
responsibilities.
 
John goes beyond the call of duty by seeking out and following up on additional 
responsibilities 
and contributes frequently to department meetings development opportunities.
 
John takes the initiative and is proactive in gathering information, as
sembling the tools or team 
members required to complete a project on time and to budget.
 
John needs to display a greater willingness to participate in team and project meetings by 
contributing more ideas and insights.
 
John has room for development of liste
ning skills particularly in team meetings when different 
viewpoints are being expressed.
 
John is always punctual and is respectful of colleagues by arriving on time for meetings.
 
John has fallen below the productivity target of $100m in revenue set in last
 
review by almost 25% below expectations at this point.
 
John is able to effectively collaborate with team

In [143]:
print(analyzer.polarity_scores(lst_str[0]))

{'neg': 0.018, 'neu': 0.878, 'pos': 0.105, 'compound': 0.886}


In [140]:
df = pd.DataFrame(lst_str, columns=['Text']) ## Converting the given text PDF file to a Pandas DataFrame

In [144]:
df.head(10)

,Text
0,\nJohn demonstrates adequate communication ski...
1,review by almost 25% below expectations at thi...
2,know how productive and results-driven you usu...
3,\nJohn exceeds expectations in the role of a ...
4,efficient effort and showed good teamwork. Eve...
5,\nJohn sets a good example by ensuring that h...
6,"upon yourself to call our supplier, research t..."
7,help us meet our overall plant producti
8,\nJohn\n-\n \nyour style of leadership has be...
9,"the same. However, I noticed that you tend to ..."


In [147]:
def result(text):
    sentiment = analyzer.polarity_scores(text)
    print(sentiment)
    neg = sentiment.get('neg')
    pos = sentiment.get('pos')
    neu = sentiment.get('neu')
    compound = sentiment.get('compound')
    return neg,pos,neu,compound

In [148]:
result('very good')

{'neg': 0.0, 'neu': 0.238, 'pos': 0.762, 'compound': 0.4927}


(0.0, 0.762, 0.238, 0.4927)

In [163]:
#Adding 3 Sentiment Specific columns and one compound score column

df['neutral'] = df['Text'].apply(lambda x:analyzer.polarity_scores(x).get('neu') )

df['negative'] = df['Text'].apply(lambda x:analyzer.polarity_scores(x).get('neg') )

df['positive'] = df['Text'].apply(lambda x:analyzer.polarity_scores(x).get('pos') )

df['compund score'] = df['Text'].apply(lambda x:analyzer.polarity_scores(x).get('compound') )

In [168]:
df.tail() 

,Text,neutral,compund score,negative,positive
44,vation of mine and would really like,0.682,0.4201,0.000,0.318
45,\nJohn Effectively solves problems rather tha...,0.667,0.6249,0.086,0.248
46,"\nworkstream. Well, keeping things is comparat...",0.837,0.9839,0.030,0.132
47,decisions until later this week.\n \n \nWhat c...,0.860,0.3818,0.000,0.140
48,\n \nJohn effectively laun\nched the go\n-\nt...,0.859,0.9186,0.018,0.123


In [201]:
df['Entities'] = df['Text'].apply(lambda x: list(nlp(x).ents)) ## Extracting Entities and adding to the dataframe

In [202]:
df.head()

,Text,neutral,compund score,negative,positive,NER,Entities
0,\nJohn demonstrates adequate communication ski...,0.878,0.8860,0.018,0.105,"[(John), (John), (John), (John), (John), (John...","[(John), (John), (John), (John), (John), (John..."
1,review by almost 25% below expectations at thi...,0.855,0.8519,0.000,0.145,"[(almost, 25, %), (John), (John)]","[(almost, 25, %), (John), (John)]"
2,know how productive and results-driven you usu...,1.000,0.0000,0.000,0.000,[],[]
3,\nJohn exceeds expectations in the role of a ...,0.785,0.9371,0.023,0.192,"[(Samuel), (John, \n \n), (Samuel)]","[(Samuel), (John, \n \n), (Samuel)]"
4,efficient effort and showed good teamwork. Eve...,0.529,0.8658,0.000,0.471,[],[]


In [204]:
df.drop('NER', inplace=True, axis=1)

In [205]:
df.head()

,Text,neutral,compund score,negative,positive,Entities
0,\nJohn demonstrates adequate communication ski...,0.878,0.8860,0.018,0.105,"[(John), (John), (John), (John), (John), (John..."
1,review by almost 25% below expectations at thi...,0.855,0.8519,0.000,0.145,"[(almost, 25, %), (John), (John)]"
2,know how productive and results-driven you usu...,1.000,0.0000,0.000,0.000,[]
3,\nJohn exceeds expectations in the role of a ...,0.785,0.9371,0.023,0.192,"[(Samuel), (John, \n \n), (Samuel)]"
4,efficient effort and showed good teamwork. Eve...,0.529,0.8658,0.000,0.471,[]


### Keyword Extraction

In [209]:
from string import punctuation
def get_keywords(text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN'] ##Keywords to be considered
    doc = nlp(text.lower()) 
    for token in doc:
        
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        
        if(token.pos_ in pos_tag):
            result.append(token.text)
                
    return result 

In [210]:
df['Keywords'] = df['Text'].apply(lambda x: list(get_keywords(x)))

In [211]:
df.head()

,Text,neutral,compund score,negative,positive,Entities,Keywords
0,\nJohn demonstrates adequate communication ski...,0.878,0.8860,0.018,0.105,"[(John), (John), (John), (John), (John), (John...","[john, adequate, communication, skills, tasks,..."
1,review by almost 25% below expectations at thi...,0.855,0.8519,0.000,0.145,"[(almost, 25, %), (John), (John)]","[expectations, point, john, able, team, projec..."
2,know how productive and results-driven you usu...,1.000,0.0000,0.000,0.000,[],"[productive, results]"
3,\nJohn exceeds expectations in the role of a ...,0.785,0.9371,0.023,0.192,"[(Samuel), (John, \n \n), (Samuel)]","[john, expectations, role, sales, associate, p..."
4,efficient effort and showed good teamwork. Eve...,0.529,0.8658,0.000,0.471,[],"[efficient, effort, good, teamwork, thing, acc..."


# Emotion Analysis

In [455]:
emotion = pd.read_csv('Emotion.csv', encoding='unicode_escape') ## reading the emotion.csv files containing about 3000 labelled text data

In [456]:
emotion.head()

,id,Text,Emotion,confidence
0,30000,Just got back from seeing @GaryDelaney in Burs...,joy,0.98
1,30001,Oh dear an evening of absolute hilarity I don'...,joy,0.958
2,30002,Been waiting all week for this game ?????? #ch...,joy,0.94
3,30003,"@gardiner_love : Thank you so much, Gloria! Yo...",joy,0.938
4,30004,I feel so blessed to work with the family that...,joy,0.938


In [457]:
emotion.drop('id', inplace=True, axis=1)

In [458]:
emotion.head()

,Text,Emotion,confidence
0,Just got back from seeing @GaryDelaney in Burs...,joy,0.98
1,Oh dear an evening of absolute hilarity I don'...,joy,0.958
2,Been waiting all week for this game ?????? #ch...,joy,0.94
3,"@gardiner_love : Thank you so much, Gloria! Yo...",joy,0.938
4,I feel so blessed to work with the family that...,joy,0.938


In [459]:
emotion['Emotion'] = emotion['Emotion'].map({'joy':0, 'anger':1, 'fear':2, 'sadness':3})

In [460]:
emotion.isnull().count()

Text          2744
Emotion       2744
confidence    2744
dtype: int64

In [466]:
emotion.dropna(inplace=True)

In [467]:
emotion.shape

(2472, 3)

In [468]:
emotion['Emotion'].value_counts()

2.0    856
0.0    783
3.0    751
1.0     82
Name: Emotion, dtype: int64

In [469]:
emotion.isna()

,Text,Emotion,confidence
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
2739,False,False,False
2740,False,False,False
2741,False,False,False
2742,False,False,False


### Now this is a multi class classification problem
* 0 = Joy
* 1 = Anger
* 2 = Fear
* 3 = Sadness

In [470]:
emotion.tail(20)

,Text,Emotion,confidence
2720,Happiness is always there. You just have to ch...,3.0,0.167
2721,Good morning chirpy #SpringEquinox and your pe...,3.0,0.167
2722,also who has amazon prime that would like to h...,3.0,0.167
2723,I added Paul Walker on Xbox but he just spends...,3.0,0.167
2724,"Nice to see Balotelli back to his best, good p...",3.0,0.167
2725,@KatieBurnett ha freshers blues would love to ...,3.0,0.167
2728,"Please bear with me, I'm not Twitter savvy?? i...",3.0,0.146
2729,@RiRiMathew even when we're fighting I'm laugh...,3.0,0.146
2730,"@SquidWasTaken Pfft- Yeye, i'm a sucker for br...",3.0,0.146
2731,Never a dull moment when talking to Nell ?????...,3.0,0.146


In [471]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

In [472]:
type(emotion['Text'][0])

str

In [473]:
ps = PorterStemmer()
def clean_text(text):
    review = re.sub('[^a-zA-Z]', ' ', text)
    review = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",review).split()) #to remove urls, user profiles and puncuations
    
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    
    return review

In [475]:
clean_text('The following regex just strips of an URL (not just http), any punc') #tesing

'follow regex strip url http punc'

In [476]:
emotion['Text'] = emotion['Text'].apply(clean_text)

In [477]:
emotion['Emotion'].value_counts()

2.0    856
0.0    783
3.0    751
1.0     82
Name: Emotion, dtype: int64

In [478]:
X = emotion.Text
y = emotion.Emotion

In [479]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [480]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

In [481]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [482]:
nb.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [483]:
X_train[0]

'got back see garydelaney burslem amaz face still hurt laugh much hilari'

In [484]:
y_train[0]

0.0

In [485]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
y_pred = nb.predict(X_test)

In [486]:
X_test

1176       never find exact emoji exact moment need panic
2402    jwonder macadamionnut well cours progress sake...
1885              georgia tech secondari soft marshmallow
1714    watch tna first time long time hell happen har...
378     anoth joy encount tribez amp castlez met mouch...
                              ...                        
1063    bloodi hell pam calm could sworn someth black ...
2622    darryella chip get dull nail gone look perfect...
607                                   blake live flawless
2581                         xxnogard g welcom dark honey
2415    higginbotham flippant dismiss worri around car...
Name: Text, Length: 742, dtype: object

In [513]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred)) ## Achieving 80% accuracy

accuracy 0.807277628032345
              precision    recall  f1-score   support

         0.0       0.88      0.85      0.86       234
         1.0       0.00      0.00      0.00        25
         2.0       0.80      0.86      0.83       267
         3.0       0.74      0.79      0.77       216

    accuracy                           0.81       742
   macro avg       0.61      0.62      0.62       742
weighted avg       0.78      0.81      0.79       742



In [495]:
df.head()

,Text,neutral,compund score,negative,positive,Entities,Keywords,Emotion,Topic
0,\nJohn demonstrates adequate communication ski...,0.878,0.8860,0.018,0.105,"[(John), (John), (John), (John), (John), (John...","[john, adequate, communication, skills, tasks,...",[2.0],3
1,review by almost 25% below expectations at thi...,0.855,0.8519,0.000,0.145,"[(almost, 25, %), (John), (John)]","[expectations, point, john, able, team, projec...",[2.0],0
2,know how productive and results-driven you usu...,1.000,0.0000,0.000,0.000,[],"[productive, results]",[2.0],6
3,\nJohn exceeds expectations in the role of a ...,0.785,0.9371,0.023,0.192,"[(Samuel), (John, \n \n), (Samuel)]","[john, expectations, role, sales, associate, p...",[0.0],3
4,efficient effort and showed good teamwork. Eve...,0.529,0.8658,0.000,0.471,[],"[efficient, effort, good, teamwork, thing, acc...",[0.0],1


In [494]:
df['Emotion'] = df['Text'].apply(lambda x: nb.predict([x]))

In [496]:
df['Emotion'] = df['Emotion'].apply(lambda x: x[0])

In [498]:
df['Emotion'] = df['Emotion'].map({0:'joy',1:'anger',2:'fear',3:'sadness'})

In [499]:
df.head()

,Text,neutral,compund score,negative,positive,Entities,Keywords,Emotion,Topic
0,\nJohn demonstrates adequate communication ski...,0.878,0.8860,0.018,0.105,"[(John), (John), (John), (John), (John), (John...","[john, adequate, communication, skills, tasks,...",fear,3
1,review by almost 25% below expectations at thi...,0.855,0.8519,0.000,0.145,"[(almost, 25, %), (John), (John)]","[expectations, point, john, able, team, projec...",fear,0
2,know how productive and results-driven you usu...,1.000,0.0000,0.000,0.000,[],"[productive, results]",fear,6
3,\nJohn exceeds expectations in the role of a ...,0.785,0.9371,0.023,0.192,"[(Samuel), (John, \n \n), (Samuel)]","[john, expectations, role, sales, associate, p...",joy,3
4,efficient effort and showed good teamwork. Eve...,0.529,0.8658,0.000,0.471,[],"[efficient, effort, good, teamwork, thing, acc...",joy,1


## Topic Modelling

In [500]:
dtm = tfidf.fit_transform(df['Text'])

In [501]:
dtm ## document term matrix

<49x585 sparse matrix of type '<class 'numpy.float64'>'
	with 2167 stored elements in Compressed Sparse Row format>

In [502]:
from sklearn.decomposition import LatentDirichletAllocation, NMF

In [503]:
nmf = NMF(n_components=4, random_state=42)

In [505]:
nmf.fit(dtm)

NMF(n_components=4, random_state=42)

In [506]:
for index,topic in enumerate(nmf.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['times', 'tend', 'remain', 'noticed', 'complete', 'needs', 'situations', 'example', 'improve', 'project', 'members', 'meetings', 'company', 'john', 'team']


THE TOP 15 WORDS FOR TOPIC #1
['teamwork', 'thing', 'effort', 'thanks', 'accurate', 'direct', 'unit', 'business', 'symptoms', 'solves', 'did', 'smoothly', 'launch', 'john', 'efficient']


THE TOP 15 WORDS FOR TOPIC #2
['workstream', 'accomplishment', 'payroll', 'track', 'automation', 'marketing', 'happen', 'like', 'vation', 'really', 'decisions', 'correct', 'later', 'ensure', 'week']


THE TOP 15 WORDS FOR TOPIC #3
['skills', 'works', 'following', 'contributes', 'seeking', 'demonstrates', 'department', 'duty', 'goes', 'frequently', 'meetings', 'samuel', 'john', 'responsibilities', 'additional']




In [507]:
topic_results = nmf.transform(dtm)

In [508]:
topic_results.argmax(axis=1)

array([3, 0, 1, 3, 1, 0, 0, 1, 3, 0, 2, 1, 2, 2, 1, 3, 0, 0, 0, 1, 3, 1,
       0, 0, 1, 3, 0, 2, 1, 2, 2, 1, 3, 0, 0, 0, 1, 3, 1, 0, 0, 1, 3, 0,
       2, 1, 2, 2, 1], dtype=int64)

In [509]:
df['Topic'] = topic_results.argmax(axis=1)

In [511]:
df['Topic'] = df['Topic'].map({0:'improvemnt',1:'good teamplayer',2:'competency',3:'skill'})

In [512]:
df.head()

,Text,neutral,compund score,negative,positive,Entities,Keywords,Emotion,Topic
0,\nJohn demonstrates adequate communication ski...,0.878,0.8860,0.018,0.105,"[(John), (John), (John), (John), (John), (John...","[john, adequate, communication, skills, tasks,...",fear,skill
1,review by almost 25% below expectations at thi...,0.855,0.8519,0.000,0.145,"[(almost, 25, %), (John), (John)]","[expectations, point, john, able, team, projec...",fear,improvemnt
2,know how productive and results-driven you usu...,1.000,0.0000,0.000,0.000,[],"[productive, results]",fear,good teamplayer
3,\nJohn exceeds expectations in the role of a ...,0.785,0.9371,0.023,0.192,"[(Samuel), (John, \n \n), (Samuel)]","[john, expectations, role, sales, associate, p...",joy,skill
4,efficient effort and showed good teamwork. Eve...,0.529,0.8658,0.000,0.471,[],"[efficient, effort, good, teamwork, thing, acc...",joy,good teamplayer


THE TOP 15 WORDS FOR TOPIC #0
['silent', 'times', 'noticed', 'tend', 'remain', 'needs', 'situations', 'example', 'improve', 'project', 'members', 'meetings', 'company', 'john', 'team']


THE TOP 15 WORDS FOR TOPIC #1
['plan', 'good', 'symptoms', 'solves', 'direct', 'unit', 'business', 'did', 'showed', 'thing', 'teamwork', 'accurate', 'thanks', 'effort', 'efficient']


THE TOP 15 WORDS FOR TOPIC #2
['things', 'drive', 'payroll', 'marketing', 'keeping', 'track', 'accomplishment', 'happen', 'workstream', 'automation', 'ensure', 'later', 'correct', 'decisions', 'week']


THE TOP 15 WORDS FOR TOPIC #3
['skills', 'works', 'following', 'meetings', 'contributes', 'seeking', 'department', 'duty', 'goes', 'frequently', 'demonstrates', 'samuel', 'john', 'responsibilities', 'additional']


THE TOP 15 WORDS FOR TOPIC #4
['way', 'teams', 'needs', 'need', 'make', 'sure', 'product', 'overall', 'help', 'producti', 'meet', 'plant', 'really', 'like', 'vation']


THE TOP 15 WORDS FOR TOPIC #5
['lan', 'ultimately', 'received', 'impacts', 'exposure', 'time', 'john', 'social', 'atlas', 'campaign', 'market', 'days', 'pr', 'rate', 'launch']


THE TOP 15 WORDS FOR TOPIC #6
['long', 'way', 'teams', 'need', 'needs', 'make', 'sure', 'product', 'results', 'usually', 'driven', 'know', 'productive', 'check', 'wanted']

## Text Classification & Sentiment Analysis

In [52]:
from nltk.corpus import movie_reviews

In [53]:
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [57]:
documents = [(list(movie_reviews.words(fileid)), category)
            for category in movie_reviews.categories()
            for fileid in movie_reviews.fileids(category)]

In [60]:
for category in movie_reviews.categories():
    for fileid in movie_reviews.fileids(category):
        print(movie_reviews.words(fileid), category)

['plot', ':', 'two', 'teen', 'couples', 'go', 'to', ...] neg
['the', 'happy', 'bastard', "'", 's', 'quick', 'movie', ...] neg
['it', 'is', 'movies', 'like', 'these', 'that', 'make', ...] neg
['"', 'quest', 'for', 'camelot', '"', 'is', 'warner', ...] neg
['synopsis', ':', 'a', 'mentally', 'unstable', 'man', ...] neg
['capsule', ':', 'in', '2176', 'on', 'the', 'planet', ...] neg
['so', 'ask', 'yourself', 'what', '"', '8mm', '"', '(', ...] neg
['that', "'", 's', 'exactly', 'how', 'long', 'the', ...] neg
['call', 'it', 'a', 'road', 'trip', 'for', 'the', ...] neg
['plot', ':', 'a', 'young', 'french', 'boy', 'sees', ...] neg
['best', 'remembered', 'for', 'his', 'understated', ...] neg
['janeane', 'garofalo', 'in', 'a', 'romantic', ...] neg
['and', 'now', 'the', 'high', '-', 'flying', 'hong', ...] neg
['a', 'movie', 'like', 'mortal', 'kombat', ':', ...] neg
['she', 'was', 'the', 'femme', 'in', '"', 'la', ...] neg
['john', 'carpenter', 'makes', 'b', '-', 'movies', '.', ...] neg
['i', "'", 'm',

['the', '13th', 'warrior', 'reeks', 'so', 'badly', ...] neg
['i', "'", 've', 'heard', 'it', 'called', '"', 'jaws', ...] neg
['billy', 'crystal', 'reappears', 'in', 'yet', ...] neg
['movies', 'can', 'do', 'the', 'two', 'big', 'es', ...] neg
['capsule', ':', 'an', 'insult', '.', 'a', 'stupid', ...] neg
['alexander', 'dumas', "'", 'the', 'three', ...] neg
['in', 'this', 're', '-', 'make', 'of', 'the', '1954', ...] neg
['when', 'i', 'first', 'saw', 'the', 'preview', 'for', ...] neg
['everyone', 'knows', 'that', 'old', 'rule', ',', "'", ...] neg
['this', 'movie', 'tries', 'to', 'present', 'itself', ...] neg
['it', "'", 's', 'tough', 'to', 'be', 'an', 'aspiring', ...] neg
['8mm', ',', 'written', 'by', 'seven', 'scribe', ...] neg
['your', 'first', 'clue', 'that', 'something', 'isn', ...] neg
['carry', 'on', 'at', 'your', 'convenience', 'is', ...] neg
['the', 'tagline', 'for', 'this', 'film', 'is', ':', ...] neg
['the', 'title', 'is', 'taken', 'from', 'the', ...] neg
['tomb', 'raider', '.', 'f

['this', 'was', 'the', 'last', 'carry', 'on', 'movie', ...] neg
['if', 'you', "'", 're', 'debating', 'whether', 'or', ...] neg
['suicide', 'is', 'pointless', ',', 'everyone', ...] neg
['the', 'realm', 'of', 'science', 'fiction', 'has', ...] neg
['back', 'in', 'february', 'at', 'the', 'monthly', ...] neg
['the', 'working', 'title', 'for', 'no', 'looking', ...] neg
['the', 'job', 'of', 'the', 'film', 'critic', 'is', ...] neg
['i', "'", 've', 'got', 'to', 'admit', 'it', '.', '.', ...] neg
['for', 'about', 'twenty', 'minutes', 'into', ...] neg
['every', 'now', 'and', 'then', ',', 'reviewers', ...] neg
['the', 'only', 'two', 'really', 'good', 'things', ...] neg
['silly', 'performances', 'and', 'some', 'huge', ...] neg
['so', 'i', 'went', 'and', 'saw', 'this', 'film', ...] neg
['by', 'phil', 'curtolo', '"', 'madonna', '-', ...] neg
['i', 'have', 'never', 'seen', 'a', 'man', 'so', 'in', ...] neg
['an', 'affluent', 'horse', 'breeder', "'", 's', ...] neg
['a', 'fullyloaded', 'entertainment', 'r

['okay', ',', 'bear', 'with', 'me', 'y', "'", 'all', ...] neg
['i', 'went', 'to', 'blair', 'witch', 'project', '2', ...] neg
['i', "'", 'm', 'giving', 'this', 'stinker', '.', ...] neg
['2', 'days', 'in', 'the', 'valley', 'is', 'more', ...] neg
['what', 'would', 'inspire', 'someone', 'who', ...] neg
['synopsis', ':', 'a', 'novelist', 'struggling', ...] neg
['okay', ',', 'okay', '.', 'maybe', 'i', 'wasn', "'", ...] neg
['in', 'life', ',', 'eddie', 'murphy', 'and', 'martin', ...] neg
['there', 'may', 'not', 'be', 'a', 'critic', 'alive', ...] neg
['renee', 'zellweger', 'stars', 'as', 'sonia', ',', ...] neg
['there', "'", 're', 'so', 'many', 'things', 'to', ...] neg
['don', "'", 't', 'let', 'this', 'movie', 'fool', ...] neg
['it', "'", 's', 'a', 'good', 'thing', 'most', ...] neg
['in', 'the', 'year', '2029', ',', 'captain', 'leo', ...] neg
['writing', 'a', 'screenplay', 'for', 'a', 'thriller', ...] neg
['all', 'right', ',', 'all', 'right', ',', 'we', 'get', ...] neg
['teenagers', 'have', 'a

['in', 'some', 'regards', ',', 'making', 'a', 'movie', ...] pos
['on', 'the', 'basis', 'of', 'this', 'film', 'alone', ...] pos
['it', "'", 's', 'a', 'fact', 'that', 'a', 'good', ...] pos
['there', 'exists', 'a', 'litany', 'of', 'differences', ...] pos
['jay', 'and', 'silent', 'bob', 'strike', 'back', ',', ...] pos
['damn', 'those', 'trailers', '.', 'had', 'it', 'not', ...] pos
['bob', 'the', 'happy', 'bastard', "'", 's', 'quickie', ...] pos
['this', 'sunday', 'afternoon', 'i', 'had', 'the', ...] pos
['a', 'common', 'complaint', 'amongst', 'film', ...] pos
['whew', '.', 'this', 'film', 'oozes', 'energy', ',', ...] pos
['steven', 'spielberg', "'", 's', '"', 'amistad', ',', ...] pos
['he', 'has', 'spent', 'his', 'entire', 'life', 'in', ...] pos
['being', 'that', 'it', 'is', 'a', 'foreign', ...] pos
['when', 'i', 'first', 'heard', 'of', 'contact', ',', ...] pos
['you', "'", 've', 'got', 'to', 'love', 'disney', '.', ...] pos
['with', 'three', 'pre', '-', 'to', 'mid', '-', 'teen', ...] pos
[

['the', 'thought', '-', 'provoking', 'question', 'of', ...] pos
['barely', 'scrapping', 'by', 'playing', 'at', 'a', ...] pos
['it', "'", 's', 'an', 'interesting', 'premise', '.', ...] pos
['one', 'of', 'kyle', 'mclachlan', "'", 's', 'earlier', ...] pos
['i', 'want', 'to', 'correct', 'what', 'i', 'wrote', ...] pos
['of', 'the', 'major', 'horror', 'sub', '-', 'genres', ...] pos
['when', 'quentin', 'tarantino', 'made', '"', 'pulp', ...] pos
['there', "'", 's', 'some', 'movies', 'i', 'enjoy', ...] pos
['this', 'summer', ',', 'one', 'of', 'the', 'most', ...] pos
['towards', 'the', 'middle', 'of', '"', 'the', 'sweet', ...] pos
['losing', 'a', 'job', 'is', 'not', 'an', 'all', 'too', ...] pos
['as', 'much', 'as', 'i', 'wanted', 'to', 'like', ...] pos
['the', 'postman', 'delivers', 'but', 'not', 'first', ...] pos
['there', 'are', 'certain', 'people', 'in', 'the', ...] pos
['i', 'guess', 'it', "'", 's', 'a', 'credit', 'to', ...] pos
['some', 'critics', ',', 'including', 'siskel', '&', ...] pos
[

['note', ':', 'some', 'may', 'consider', 'portions', ...] pos
['robocop', 'is', 'an', 'intelligent', 'science', ...] pos
['probably', 'the', 'most', 'popular', 'and', ...] pos
['cinema', 'has', 'been', 'around', 'for', 'about', ...] pos
['it', "'", 's', 'no', 'secret', 'in', 'the', 'motion', ...] pos
['when', 'andy', 'leaves', 'for', 'cowboy', 'camp', ...] pos
['david', 'lynch', "'", 's', '"', 'blue', 'velvet', ...] pos
['if', 'chris', 'farley', 'had', 'strapped', 'some', ...] pos
['"', 'you', 'leave', 'little', 'notes', 'on', 'my', ...] pos
['ingredients', ':', 'lost', 'parrot', 'trying', 'to', ...] pos
['tempe', 'mills', 'cinema', ',', 'az', '--', 'this', ...] pos
['there', 'is', 'nothing', 'like', 'american', ...] pos
['john', 'von', 'neumann', ',', 'progenitor', 'of', ...] pos
['as', 'the', 'small', 'boats', 'rock', 'slowly', ...] pos
['"', 'the', 'faculty', ',', '"', 'the', 'heavily', ...] pos
['the', 'central', 'focus', 'of', 'michael', ...] pos
['after', 'the', 'average', 'mouse

In [64]:
## Taking the top 3000 most frequent words

random.shuffle(documents)
all_words = []

for w in movie_reviews.words():
    all_words.append(w.lower())
    
all_words = nltk.FreqDist(all_words)

word_features = list(all_words.keys())[:3000]

In [69]:
word_features

['plot',
 ':',
 'two',
 'teen',
 'couples',
 'go',
 'to',
 'a',
 'church',
 'party',
 ',',
 'drink',
 'and',
 'then',
 'drive',
 '.',
 'they',
 'get',
 'into',
 'an',
 'accident',
 'one',
 'of',
 'the',
 'guys',
 'dies',
 'but',
 'his',
 'girlfriend',
 'continues',
 'see',
 'him',
 'in',
 'her',
 'life',
 'has',
 'nightmares',
 'what',
 "'",
 's',
 'deal',
 '?',
 'watch',
 'movie',
 '"',
 'sorta',
 'find',
 'out',
 'critique',
 'mind',
 '-',
 'fuck',
 'for',
 'generation',
 'that',
 'touches',
 'on',
 'very',
 'cool',
 'idea',
 'presents',
 'it',
 'bad',
 'package',
 'which',
 'is',
 'makes',
 'this',
 'review',
 'even',
 'harder',
 'write',
 'since',
 'i',
 'generally',
 'applaud',
 'films',
 'attempt',
 'break',
 'mold',
 'mess',
 'with',
 'your',
 'head',
 'such',
 '(',
 'lost',
 'highway',
 '&',
 'memento',
 ')',
 'there',
 'are',
 'good',
 'ways',
 'making',
 'all',
 'types',
 'these',
 'folks',
 'just',
 'didn',
 't',
 'snag',
 'correctly',
 'seem',
 'have',
 'taken',
 'pretty',


In [67]:
def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
        
    return features 

In [68]:
print((find_features(movie_reviews.words('neg/cv000_29416.txt'))))

{'plot': True, ':': True, 'two': True, 'teen': True, 'couples': True, 'go': True, 'to': True, 'a': True, 'church': True, 'party': True, ',': True, 'drink': True, 'and': True, 'then': True, 'drive': True, '.': True, 'they': True, 'get': True, 'into': True, 'an': True, 'accident': True, 'one': True, 'of': True, 'the': True, 'guys': True, 'dies': True, 'but': True, 'his': True, 'girlfriend': True, 'continues': True, 'see': True, 'him': True, 'in': True, 'her': True, 'life': True, 'has': True, 'nightmares': True, 'what': True, "'": True, 's': True, 'deal': True, '?': True, 'watch': True, 'movie': True, '"': True, 'sorta': True, 'find': True, 'out': True, 'critique': True, 'mind': True, '-': True, 'fuck': True, 'for': True, 'generation': True, 'that': True, 'touches': True, 'on': True, 'very': True, 'cool': True, 'idea': True, 'presents': True, 'it': True, 'bad': True, 'package': True, 'which': True, 'is': True, 'makes': True, 'this': True, 'review': True, 'even': True, 'harder': True, 'wri

In [70]:
featuresets = [(find_features(rev), category) for (rev, category) in documents]

In [78]:
len(featuresets)

2000

## Building Naive Bayes Classifier

In [79]:
training_set = featuresets[:1900]
testing_set = featuresets[1900:]

In [83]:
# training_set

In [80]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [81]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

Classifier accuracy percent: 79.0


In [84]:
classifier.show_most_informative_features(15)

Most Informative Features
                   sucks = True              neg : pos    =     10.1 : 1.0
                  annual = True              pos : neg    =      9.8 : 1.0
                bothered = True              neg : pos    =      9.6 : 1.0
                 frances = True              pos : neg    =      9.1 : 1.0
             silverstone = True              neg : pos    =      7.6 : 1.0
              schumacher = True              neg : pos    =      7.3 : 1.0
                  regard = True              pos : neg    =      7.1 : 1.0
                    mena = True              neg : pos    =      6.9 : 1.0
                  shoddy = True              neg : pos    =      6.9 : 1.0
                  suvari = True              neg : pos    =      6.9 : 1.0
           unimaginative = True              neg : pos    =      6.9 : 1.0
               atrocious = True              neg : pos    =      6.5 : 1.0
                 idiotic = True              neg : pos    =      6.5 : 1.0

## Saving the Classifier

In [85]:
import pickle

In [86]:
save_classifier = open("naivebayes.pickle","wb")
pickle.dump(classifier, save_classifier)

In [87]:
save_classifier.close()

In [88]:
classifier_f = open("naivebayes.pickle", "rb")
classifier = pickle.load(classifier_f)
classifier_f.close()